In [3]:
!pip install "git+https://github.com/jemisjoky/TorchMPS.git"
!pip install "git+https://github.com/rballester/tntorch.git"
import tntorch as tn
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
!pip install torchmetrics
from torchmetrics.classification import MulticlassAccuracy
from torchmps import MPS

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/jemisjoky/TorchMPS.git to c:\users\piche\appdata\local\temp\pip-req-build-ndu3qjyo
  Resolved https://github.com/jemisjoky/TorchMPS.git to commit 6c0bc1a8e2c15acba8570ca9ffe2b4a0c7135165
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/jemisjoky/TorchMPS.git 'C:\Users\piche\AppData\Local\Temp\pip-req-build-ndu3qjyo'


^C
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/rballester/tntorch.git to c:\users\piche\appdata\local\temp\pip-req-build-gsb24hy9
  Resolved https://github.com/rballester/tntorch.git to commit 241bf7ad2b806f6677a5e23534247f35f3a70f10
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/rballester/tntorch.git 'C:\Users\piche\AppData\Local\Temp\pip-req-build-gsb24hy9'


Defaulting to user installation because normal site-packages is not writeable


We shall choose the Hyperparameters in our setup

In [4]:
# Data hyperparameters
nb_train = 20
nb_test = 5
chosen_bs = 150

# Student training parameters
# MPS parameters
bond_dim = 20
adaptive_mode = False
periodic_bc = False
gn_var = 0.3 #added gaussian noise variance
gn_mean = 0 #added gaussian noise mean
n_epochs_lmps = 18
mps_learn_rate = 0.0001
mps_reg = 0.0
student_chosen_loss = nn.CrossEntropyLoss().to(chosen_device)

NameError: name 'chosen_device' is not defined

In [ ]:
# Import the mnist dataset
train_set = torchvision.datasets.MNIST(root = './datasets', train = True,    transform = transforms.ToTensor(),  download = True )

train_subset = torch.utils.data.SubsetRandomSampler(range(nb_train))

train_iterator = torch.utils.data.DataLoader(dataset = train_set, sampler = train_subset, batch_size=chosen_bs)


test_set = torchvision.datasets.MNIST(root = './datasets', train = False, transform = transforms.ToTensor(),  download = True)

test_subset = torch.utils.data.SubsetRandomSampler(range(nb_test))

test_iterator = torch.utils.data.DataLoader(dataset = test_set, sampler = test_subset, batch_size = chosen_bs)

In [ ]:

class LMPS_patcher(nn.Module):
    def __init__(self):
        super(LMPS_patcher, self).__init__()
        mps_student = MPS(
            input_dim=28 ** 2,
            output_dim=10,
            bond_dim=bond_dim,
            adaptive_mode=adaptive_mode,
            periodic_bc=periodic_bc,
        )

    def forward(self, mps_student):
        y = self.mps_student(x)
        return y

student = LMPS_patcher()

# Instantiate the optimizer and softmax
student_optimizer = torch.optim.Adam(student.parameters(), lr = mps_learn_rate,
                                  weight_decay = mps_reg)
softmax = nn.Softmax(dim=1)

# Training loop 
for epoch in range(n_epochs_lmps):
    for (x_mb, _) in train_iterator:
        # Reshape, add gaussian noise, put on the chosen device
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        x_mb = (x_mb + torch.randn(size=x_mb.size())).to(chosen_device)

        # Get softmax of output of teacher
        y_mb = (softmax(fcnn_teacher(x_mb))).to(chosen_device)

        # Foward propagation
        y_hat_mb = student(x_mb)
        loss = student_chosen_loss(y_hat_mb, y_mb)

        # Backpropagation
        loss.backward()
        lmps_optimizer.step()
        lmps_optimizer.zero_grad()

    print(loss.item())


# get accuracy
for (x_mb, y_mb) in test_iterator:
    print("testing teacher")
    print(MulticlassAccuracy(num_classes=10)(mps_student(x_mb.reshape(-1, 784)), y_mb))
